In [1]:
from audiolabel import read_label
from os.path import abspath,join
import pandas as pd
import numpy as np
import os

In [2]:
datadir = abspath('output/')

files = [fileName for fileName in os.listdir(datadir) if fileName[-8:]=='TextGrid']

df1 = pd.DataFrame()
df2 = pd.DataFrame()

for file in files:
    tg_file = join(datadir,file)
    [wd1df,ph1df,wd2df,ph2df,stop1df,stop2df,vot1df,vot2df] = read_label(
        tg_file, 
        'praat' 
        # tiers=[language.lower()+' - words', language.lower()+' - phones', 'stops', 'AutoVOT']
        )
    whole1df = pd.merge_asof(
            ph1df.drop('fname', axis = 'columns') \
                .rename({'t1':'t1_ph', 't2':'t2_ph', 'label':'phone'}, axis = 'columns'),
            wd1df.drop('fname', axis = 'columns') \
                .rename({'t1':'t1_wd', 't2':'t2_wd', 'label':'word'}, axis = 'columns'),
            left_on = 't1_ph',
            right_on = 't1_wd'
        )
    whole1df = pd.merge_asof(
            whole1df,
            stop1df.drop('fname', axis = 'columns') \
                .rename({'t1':'t1_stop', 't2':'t2_stop', 'label':'stop'}, axis = 'columns'),
            left_on = 't1_ph',
            right_on = 't1_stop'
        )
    whole1df = pd.merge_asof(
        whole1df,
        vot1df.rename({'t1':'t1_vot', 't2':'t2_vot', 'label':'vot'}, axis = 'columns'),
        left_on = 't1_ph',
        right_on = 't1_vot',
        direction = 'nearest'
    )
    whole1df['speaker'] = "1"
    df1 = df1.append(whole1df, ignore_index = True)
    
    whole2df = pd.merge_asof(
            ph2df.drop('fname', axis = 'columns') \
                .rename({'t1':'t1_ph', 't2':'t2_ph', 'label':'phone'}, axis = 'columns'),
            wd2df.drop('fname', axis = 'columns') \
                .rename({'t1':'t1_wd', 't2':'t2_wd', 'label':'word'}, axis = 'columns'),
            left_on = 't1_ph',
            right_on = 't1_wd'
        )
    whole2df = pd.merge_asof(
            whole2df,
            stop2df.drop('fname', axis = 'columns') \
                .rename({'t1':'t1_stop', 't2':'t2_stop', 'label':'stop'}, axis = 'columns'),
            left_on = 't1_ph',
            right_on = 't1_stop'
        )
    whole2df = pd.merge_asof(
        whole2df,
        vot2df.rename({'t1':'t1_vot', 't2':'t2_vot', 'label':'vot'}, axis = 'columns'),
        left_on = 't1_ph',
        right_on = 't1_vot',
        direction = 'nearest'
    )
    whole2df['speaker'] = "2"
    df2 = df2.append(whole2df, ignore_index = True)
    combined_df = pd.concat([df1,df2])
    
df = combined_df.copy()
df['vot'] = df['t2_vot'] - df['t1_vot']

In [3]:
countdf = df.groupby(['t1_wd']).size()
countdf = pd.DataFrame({'t1_wd':countdf.index, 'ph_per_wd':countdf.values})

df = df.merge(countdf, on= 't1_wd')

df['wd_dur'] = df['t2_wd'] - df['t1_vot']
df

,t1_ph,t2_ph,phone,t1_wd,t2_wd,word,t1_stop,t2_stop,stop,t1_vot,t2_vot,vot,fname,speaker,ph_per_wd,wd_dur
0,0.00000,2.000000,,0.000000,2.000000,,0.000000,23.721125,,0.000,23.788000,23.788000,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,8,2.000000
1,0.00000,2.000000,,0.000000,2.000000,,0.000000,66.938020,,0.000,66.950000,66.950000,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,8,2.000000
2,0.00000,2.000000,,0.000000,2.000000,,0.000000,54.644950,,0.000,54.691000,54.691000,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,8,2.000000
3,0.00000,2.000000,,0.000000,2.000000,,0.000000,8.498750,,0.000,8.536000,8.536000,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,8,2.000000
4,0.00000,2.000000,,0.000000,2.000000,,0.000000,17.542000,,0.000,17.616000,17.616000,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,8,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13904,303.68000,303.770000,K,300.819454,303.767488,!SIL,270.716125,306.341224,,270.719,306.341224,35.622224,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,2,33.048488
13905,303.77000,303.830000,UW,303.767488,303.930000,cool,270.716125,306.341224,,270.719,306.341224,35.622224,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,2,33.211000
13906,303.83000,303.930000,L,303.767488,303.930000,cool,270.716125,306.341224,,270.719,306.341224,35.622224,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,2,33.211000
13907,303.93000,304.211060,!SIL,303.930000,304.341224,!SIL,270.716125,306.341224,,270.719,306.341224,35.622224,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,2,33.622224


In [4]:
df.replace("", np.nan, inplace=True)
df.dropna(subset=['stop','vot'], inplace=True)

In [5]:
df

,t1_ph,t2_ph,phone,t1_wd,t2_wd,word,t1_stop,t2_stop,stop,t1_vot,t2_vot,vot,fname,speaker,ph_per_wd,wd_dur
48,23.721125,23.851125,K,23.721125,24.141125,can,23.721125,23.851125,K,23.788,23.863,0.075,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,3,0.353125
63,35.680750,35.780750,K,35.680750,35.978775,kid,35.680750,35.780750,K,35.729,35.792,0.063,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,3,0.249775
176,70.424875,70.594875,K,70.424875,70.935409,comma,70.424875,70.594875,K,70.525,70.600,0.075,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,4,0.410409
489,169.962750,170.082750,K,169.962750,170.232750,kind,169.962750,170.082750,K,170.037,170.079,0.042,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,4,0.195750
502,171.669375,171.769375,K,171.669375,171.899375,can,171.669375,171.769375,K,171.729,171.770,0.041,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,3,0.170375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13664,200.410250,200.550250,T,200.410250,200.800092,tree,200.410250,200.550250,T,200.425,200.559,0.134,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,3,0.375092
13758,226.828125,226.968125,K,226.828125,227.332995,cat,226.828125,226.968125,K,226.895,226.978,0.083,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,4,0.437995
13770,236.727500,236.957500,T,236.727500,237.157500,two,236.727500,236.957500,T,236.788,236.954,0.166,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,2,0.369500
13828,261.546250,261.776250,T,261.546250,261.976250,tree,261.546250,261.776250,T,261.640,261.798,0.158,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,3,0.336250


In [6]:
def speakerType(row):
    if row['fname'] == "/Users/ernesto/Desktop/Projects/vot-cp/vot-dev/output/DP_CHF_04_KOF_03_EN_KOF_03_DP_CHF_04_KOF_03_EN_CHF_04_output.TextGrid":
        if row["speaker"] == "1":
            return "chinese"
        else:
            return "korean"
    elif row['fname'] == "/Users/ernesto/Desktop/Projects/vot-cp/vot-dev/output/DP_TH_01_KO_05_EN_TH_01_DP_TH_01_KO_05_EN_KO_05_output.TextGrid":
        if row["speaker"] == "1":
            return "thai"
        else:
            return "korean"
    elif row['fname'] == "/Users/ernesto/Desktop/Projects/vot-cp/vot-dev/output/DP_IT_01_SP_02_EN_IT_01_DP_IT_01_SP_02_EN_SP_02_output.TextGrid":
        if row["speaker"] == "1":
            return "italian"
        else:
            return "spanish"
    else:
        return "english"

In [7]:
df["speakerLang"] = df.apply(lambda row: speakerType(row), axis=1)

In [8]:
df

,t1_ph,t2_ph,phone,t1_wd,t2_wd,word,t1_stop,t2_stop,stop,t1_vot,t2_vot,vot,fname,speaker,ph_per_wd,wd_dur,speakerLang
48,23.721125,23.851125,K,23.721125,24.141125,can,23.721125,23.851125,K,23.788,23.863,0.075,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,3,0.353125,italian
63,35.680750,35.780750,K,35.680750,35.978775,kid,35.680750,35.780750,K,35.729,35.792,0.063,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,3,0.249775,italian
176,70.424875,70.594875,K,70.424875,70.935409,comma,70.424875,70.594875,K,70.525,70.600,0.075,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,4,0.410409,italian
489,169.962750,170.082750,K,169.962750,170.232750,kind,169.962750,170.082750,K,170.037,170.079,0.042,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,4,0.195750,italian
502,171.669375,171.769375,K,171.669375,171.899375,can,171.669375,171.769375,K,171.729,171.770,0.041,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,1,3,0.170375,italian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13664,200.410250,200.550250,T,200.410250,200.800092,tree,200.410250,200.550250,T,200.425,200.559,0.134,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,3,0.375092,english
13758,226.828125,226.968125,K,226.828125,227.332995,cat,226.828125,226.968125,K,226.895,226.978,0.083,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,4,0.437995,english
13770,236.727500,236.957500,T,236.727500,237.157500,two,236.727500,236.957500,T,236.788,236.954,0.166,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,2,0.369500,english
13828,261.546250,261.776250,T,261.546250,261.976250,tree,261.546250,261.776250,T,261.640,261.798,0.158,/Users/ernesto/Desktop/Projects/vot-cp/vot-dev...,2,3,0.336250,english


In [9]:
export_csv = df.to_csv('results.csv', index=None, header=True)